<a href="https://colab.research.google.com/github/lukezhq10/Book-Recommendation-Engine-using-KNN/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-07-07 01:48:29--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.4’

book-crossings.zip. 100%[===================>]  24.88M   122MB/s    in 0.2s    

2023-07-07 01:48:29 (122 MB/s) - ‘book-crossings.zip.4’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

df_books.head()


,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [ ]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [ ]:
df_books.shape

(271379, 3)

In [ ]:
df_ratings.shape

(1149780, 3)

# Cleaning the Data


In [ ]:
# merge dataframes
df = df_ratings

df = pd.merge(right = df, left = df_books, on='isbn')

df.head()

,isbn,title,author,user,rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2,0.0
1,0002005018,Clara Callan,Richard Bruce Wright,8,5.0
2,0002005018,Clara Callan,Richard Bruce Wright,11400,0.0
3,0002005018,Clara Callan,Richard Bruce Wright,11676,8.0
4,0002005018,Clara Callan,Richard Bruce Wright,41385,0.0


In [ ]:
# check null data
df.isnull().sum()

isbn      0
title     0
author    1
user      0
rating    0
dtype: int64

In [ ]:
# drop null data
df.dropna(inplace=True)
df.isnull().sum()

isbn      0
title     0
author    0
user      0
rating    0
dtype: int64

In [ ]:
# remove users with <200 ratings

# count number of ratings per user
user_rating_counts = df['user'].value_counts()

# get list of users with <200 ratings
users_with_200_ratings = user_rating_counts[user_rating_counts < 200].index

# filter df based on selected users
df_filtered = df[~df['user'].isin(users_with_200_ratings)]

df_filtered.shape

(475020, 5)

In [ ]:
# remove books with <100 ratings

# count number of ratings per user
book_rating_counts = df['isbn'].value_counts()

# get list of books with <100 ratings
books_with_100_ratings = book_rating_counts[book_rating_counts < 100].index

# filter df based on selected ratings
df_filtered = df_filtered[~df_filtered['isbn'].isin(books_with_100_ratings)]

df_filtered.shape

(48358, 5)

In [ ]:
# drop duplicates
df_filtered = df_filtered.drop_duplicates(['title', 'user'])
df_filtered.shape

(47978, 5)

In [ ]:
df_filtered.head()

,isbn,title,author,user,rating
103,0440234743,The Testament,John Grisham,2977,0.0
105,0440234743,The Testament,John Grisham,3363,0.0
108,0440234743,The Testament,John Grisham,7346,9.0
114,0440234743,The Testament,John Grisham,11676,9.0
117,0440234743,The Testament,John Grisham,13552,8.0


# Preparing dataset for KNN

In [ ]:
pivot_table = df_filtered.pivot_table(index='title', columns='user', values='rating', fill_value=0)

In [ ]:
pivot_table.head()

user,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
1st to Die: A Novel,0,0,0,0,0,0,0,0,0,9,...,0,0,0,0,0,0,0,0,0,0
2nd Chance,0,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4 Blondes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
pivot_table.loc['The Queen of the Damned (Vampire Chronicles (Paperback))'][:5]

user
254     0
2276    0
2766    0
2977    0
3363    0
Name: The Queen of the Damned (Vampire Chronicles (Paperback)), dtype: int64

# Build model

In [ ]:
knn = NearestNeighbors(metric='cosine')
knn.fit(pivot_table.values)

NearestNeighbors(metric='cosine')

# Create get_recommends()

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(title = ""):
  # find book in pivot table
  try:
    book = pivot_table.loc[title]
  except KeyError as e:
    print('The requested book', e, 'does not exist in the database')
    return

  # get book_rating based on book title
  book_rating = np.array([pivot_table.loc[title]])

  # find nearest neighbors
  distances, indices = knn.kneighbors(book_rating, n_neighbors=5)

  recommended_books = pd.DataFrame({
      'title': pivot_table.iloc[indices[0]].index,
      'distance': distances[0]
  }) \
  .sort_values(by='distance', ascending=False) \
  .head(5).values


  return [title, recommended_books]

In [ ]:
get_recommends('The Queen of the Damned (Vampire Chronicles (Paperback))')

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 array([['The Witching Hour (Lives of the Mayfair Witches)',
         0.736278709872293],
        ['Interview with the Vampire', 0.7325513300862068],
        ['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
         0.5298544346433054],
        ['The Vampire Lestat (Vampire Chronicles, Book II)',
         0.5145133637013516],
        ['The Queen of the Damned (Vampire Chronicles (Paperback))',
         1.1102230246251565e-16]], dtype=object)]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([['The Weight of Water', 0.7678344518368971],
       ['The Surgeon', 0.7669050871763631],
       ['I Know This Much Is True', 0.764642023144184],
       ['The Lovely Bones: A Novel', 0.7158566833346476],
       ["Where the Heart Is (Oprah's Book Club (Paperback))", 0.0]],
      dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
